# LLM with Web Search and Crawl

Code to crawl the top n pages of a Google search result and serve them to LLM in order to utilize rich context.



In [2]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv(override=True) 

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-08-01-preview"
)

CHAT_COMPLETIONS_MODEL = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')

bs4 or scrapy?

In [3]:
import requests
import json
import scrapy
from bs4 import BeautifulSoup
import httpx
import asyncio
from urllib.parse import urljoin
from azure.ai.projects.models import MessageRole, BingGroundingTool
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")
BING_GROUNDING_PROJECT_CONNECTION_STRING = os.getenv("BING_GROUNDING_PROJECT_CONNECTION_STRING")
BING_GROUNDING_AGENT_ID = os.getenv("BING_GROUNDING_AGENT_ID")
BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME = os.getenv("BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME")
BING_GROUNDING_CONNECTION_NAME = os.getenv("BING_GROUNDING_CONNECTION_NAME")
# Web search mode: "google" or "bing"
# it can be changed when users want to use different search engine
WEB_SEARCH_MODE = os.getenv("WEB_SEARCH_MODE")

def extract_text_and_tables_by_bs4(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    # Extract main text
    paragraphs = [p.get_text().strip() for p in soup.find_all("p") if p.get_text().strip()]
    text = "\n".join(paragraphs)
    return text


async def extract_text_and_tables_async(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }
    async with httpx.AsyncClient(timeout=3, follow_redirects=True) as client:
        try:
            response = await client.get(url, headers=headers)
            response.raise_for_status()
        except httpx.HTTPStatusError as e:
            # Handle 302 redirect manually if follow_redirects fails
            if e.response.status_code == 302 and "location" in e.response.headers:
                redirect_url = e.response.headers["location"]
                if not redirect_url.startswith("http"):
                    # handle relative redirects
                    redirect_url = urljoin(url, redirect_url)
                try:
                    response = await client.get(redirect_url, headers=headers)
                    response.raise_for_status()
                except Exception as e2:
                    print(f"Redirect request failed: {e2}")
                    return ""
            else:
                print(f"Request failed: {e}")
                return ""
        except httpx.HTTPError as e:
            print(f"Request failed: {e}")
            return ""

        selector = scrapy.Selector(text=response.text)
        paragraphs = [p.strip() for p in selector.css('p::text').getall() if p.strip()]
        text = "\n".join(paragraphs)
        return text

async def add_context_async(top_urls = []):
    async def gather_contexts():
        tasks = [extract_text_and_tables_async(url) for url in top_urls]
        results = await asyncio.gather(*tasks)
        return results
    return await gather_contexts()

def google_search(query, num=5):
    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        "q": query,
        "key": GOOGLE_API_KEY,
        "cx": GOOGLE_CSE_ID,
        "num": num, 
        "locale": "ko",  # 한국어로 검색
        "siteSearch": "samsung.com",
        "siteSearchFilter": "e",
    }
    response = requests.get(url, params=params)
    results = response.json()
    return results.get("items", [])

def bing_grounding_search(query, num=5, search_type="web"):
    try:
        creds = DefaultAzureCredential()
        
        project_client = AIProjectClient.from_connection_string(
            credential=creds,
            conn_str=BING_GROUNDING_PROJECT_CONNECTION_STRING,
        )
        
        agent_id = BING_GROUNDING_AGENT_ID
        
        if not agent_id:
            print("BING_GROUNDING_AGENT_ID is not set. Create new agent...")
            connection_name = BING_GROUNDING_CONNECTION_NAME
            
            bing_connection = project_client.connections.get(
                connection_name=connection_name,
            )
            conn_id = bing_connection.id
            
            bing = BingGroundingTool(connection_id=conn_id)
            
            
            agent = project_client.agents.create_agent(
                model=BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME,
                name="temporary-bing-agent",
                instructions="You are a helpful assistant that searches the web",
                tools=bing.definitions,
                headers={"x-ms-enable-preview": "true"}
            )
            agent_id = agent.id
            print(f"New agent created. Agent ID: {agent_id}")
        else:
            print(f"Existing agent ID: {agent_id}")
            try:
                agent = project_client.agents.get_agent(agent_id)
            except Exception as agent_error:
                print(f"Failed to retrieve agent: {agent_error}")
                return []

        thread = project_client.agents.create_thread()
        
        message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=f"Search the web for: {query}. Return only the top {num} most relevant results as a list.",
        )

        print(f"Message created, ID: {message.id}")

        run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
        
        if run.status == "failed":
            print(f"Execution failed: {run.last_error}")
            return []
        print(f"Run completed successfully. Status: {run.status}")
        results = []
        response_message = project_client.agents.list_messages(thread_id=thread.id).get_last_message_by_role(
            MessageRole.AGENT
        )
        if response_message.url_citation_annotations:
            # Extract content text and annotations
            if response_message.content:
                for content_item in response_message["content"]:
                    if content_item["type"] == "text":
                        text_content = content_item["text"]["value"]
                        print("Extracted Text Content:")
                        print(text_content)
            for annotation in response_message.url_citation_annotations:
                if annotation["type"] == "url_citation":
                    url_citation = annotation["url_citation"]
                    url = url_citation["url"]
                    title = url_citation["title"]
                    # set the results same as google json format
                    results.append({"link": url, "title": title})

        if not BING_GROUNDING_AGENT_ID and 'agent' in locals() and hasattr(agent, 'id'):
            try:
                project_client.agents.delete_agent(agent.id)
                
            except Exception as delete_error:
                print(f"Error deleting agent: {delete_error}")

        return results if results else []
    except Exception as e:
        print(f"Bing Grounding error : {e}")
        return []

def web_search(query, num=5, search_type="web"):
    """환경 변수에 따라 Google Search API 또는 Bing Grounding을 사용하여 검색 수행"""
    
    if WEB_SEARCH_MODE == "bing":
        print(f"Bing Grounding 검색 사용: {query}")
        try:
            return bing_grounding_search(query, num, search_type)
            
        except Exception as e:
            print(f"Bing Grounding 검색 중 오류 발생: {e}")
    else:
        print(f"Google Search API 사용: {query}")
        return google_search(query, num, search_type)

       
QUERY_REWRITE_PROMPT = """
            <<지시문>>
            너는 구글 검색과 LLM 질의 최적화 전문가야. 사용자가 입력한 질문을 두 가지 목적에 맞게 재작성해.

            1. Web Search용 Query Rewrite:
            - 사용자의 질문을 실제 검색 엔진 검색창에 입력할 수 있도록, 명확하고 간결한 핵심 키워드 중심의 검색어로 재작성해.
            - 불필요한 문장, 맥락 설명은 빼고, 검색에 최적화된 형태로 만들어.
            - 핵심 키워드를 반복적으로 사용해 검색의 정확도를 높여.

            2. LLM Query용 Rewrite:
            - 사용자의 질문을 LLM이 더 잘 이해하고 답변할 수 있도록, 맥락과 의도를 명확히 드러내는 자연스러운 문장으로 재작성해.
            - 필요한 경우 추가 설명이나 세부 조건을 포함해서 질문의 목적이 분명히 드러나도록 만들어.
            - LLM이 답변에 집중할 수 있도록 핵심 단어를 반복 사용해.

            <<예시>>
            * 질문: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘
            * 웹 검색용 재작성: 삼성전자 3구 이상 인덕션 추천
            * LLM 답변용 재작성: 삼성전자 인덕션 중 2구 모델이 아닌, 3구 이상 또는 다양한 화구 수를 가진 다른 인덕션 제품을 추천해 주세요. 각 모델의 주요 기능과 장점도 함께 알려주세요.

            <<질문>>
            {user_query}

            <<출력포맷>>
            반드시 아래와 같이 json 형식으로 출력해.
            {"web_search": "웹 검색용 재작성", "llm_query": "LLM 답변용 재작성"}
        """     
  
def rewrite_query_for_search_and_llm(query, client: AzureOpenAI):
        response = client.chat.completions.create(
            model=CHAT_COMPLETIONS_MODEL,
            messages=[
                {"role": "system", "content": QUERY_REWRITE_PROMPT},
                {"role": "user", "content": query}
            ],
            temperature=0.8,
            max_tokens=300,
            response_format= {"type": "json_object"},
        )
        
        return json.loads(response.choices[0].message.content.strip())







In [ ]:
from IPython.display import Markdown, display
from datetime import datetime
import time

RESULTS_COUNT = 3

inputs = [
    "삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘",
    "부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘",
    "삼성전자 25년 제품이 작년 대비 좋아진것은",
    "삼성전자 JBL과 하만카돈 차이점이 뭐야",
    "갤럭시 버즈 이어버드 한쪽을 새로 구매했는데 페어링 어떻게 하나요",
    "삼성전자 S25 무게가 S24와 비교 했을때 얼마나 차이나"
]

#TODO 날씨나 뉴스, 기타 다른 특정정보는 Function Call
# inputs = ["날씨, 뉴스"] ##

async def process_web_search_call(RESULTS_COUNT, input):
    
    start_time = time.time()
    
    print(f"Original Input: {input}")
    
    query_rewrite = rewrite_query_for_search_and_llm(input, client)
    print(f"Web Search Query: {query_rewrite['web_search']}")
    print(f"LLM Query: {query_rewrite['llm_query']}")

    results = web_search(query_rewrite['web_search'], RESULTS_COUNT)
    if results and isinstance(results, list) and len(results) > 0:
        print(f"Web Search Results: {len(results)}")
        top_urls = [results[i]["link"] for i in range(len(results))]
        contexts = await add_context_async(top_urls)
    else:
        print("No results found or invalid response from web_search.")
        contexts = []

    # for i, context in enumerate(contexts):
    #     print(f"Context {i+1}: {context}...")  # Print first 1000 chars of each context
    #     print("\n--- End of Context ---\n")

    now = datetime.now()
    year = now.year
    month = now.month
    day = now.day

    system_prompt = "너는 삼성전자 제품 관련 정보를 제공하는 챗봇이야. 답변은 마크다운으로 이모지를 1~2개 포함해서 작성해줘."
    user_prompt = f"""
        너는 아래 제공하는 웹검색에서 검색한 컨텍스트를 바탕으로 질문에 대한 답변을 제공해야 해. 컨텍스트를 최대한 활용하여 풍부하게 답변을 해야해. 
        현재는 {year}년 {month}월 {day}일이므로 최신의 데이터를 기반으로 답변을 해줘.
        구글에서 제공한 컨텍스트: {contexts}
        질문: {query_rewrite['llm_query']}
        """

    response = client.chat.completions.create(
        model=CHAT_COMPLETIONS_MODEL,
        messages=[{"role": "system", "content": system_prompt},
                 {"role": "user", "content": user_prompt}],
        top_p=0.9,
        max_tokens=1500
    )

    display(Markdown(response.choices[0].message.content))
    end_time = time.time()
    print(f"elapsed time: {end_time - start_time:.2f} seconds")

for input in inputs:
    await process_web_search_call(RESULTS_COUNT, input)

Original Input: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘


Web Search Query: 삼성전자 3구 이상 인덕션 추천
LLM Query: 삼성전자 인덕션 중 2구 모델이 아닌, 3구 이상 또는 다양한 화구 수를 가진 다른 인덕션 제품을 추천해 주세요. 각 모델의 주요 기능과 장점도 함께 알려주세요.
Bing Grounding 검색 사용: 삼성전자 3구 이상 인덕션 추천
Existing agent ID: asst_eY1zfMrBH0XcigQZpOrGXmXk
Message created, ID: msg_arMjU6GuHLaYmeo7yx6U9vtW
Run completed successfully. Status: RunStatus.COMPLETED
Extracted Text Content:
Here are the top 3 relevant results for recommendations regarding Samsung 3-burner induction cooktops:

1. **Samsung Bespoke Built-In Induction Overview**  
   Offers specs and reviews on Samsung Bespoke 3-burner induction products.  
   Source: castleg.tistory.com【3:1†source】.

2. **Samsung NZ63B6527XW Review**  
   Highlights a popular Samsung induction model with features and recommendations.  
   Source: hiangels.tistory.com【3:2†source】.

3. **Samsung 3-Burner Induction Models Comparison**  
   Detailed guide comparing Samsung's 3-burner induction including benefits and drawbacks.  
   Source: electrinsight.tistory.com【3:0†source】.

삼성전자의 인덕션 제품 중에서 3구 이상의 모델을 찾고 계시다면, 다음과 같은 추천 제품들을 고려해 보세요! 🥘✨

### 1. **삼성 BESPOKE 인덕션 NZ63DB657CFH**
- **화구 수**: 3구
- **주요 기능**:
  - **플렉스존**: 좌측 화구 2개를 연결하여 넓은 공간으로 사용할 수 있어 대형 조리기구 사용 시 유용합니다.
  - **AI 끓음 방지 기능**: 진동 센서를 통해 끓음을 감지하고 화력을 조절해 줍니다.
- **장점**:
  - 화이트 비스포크 디자인으로 주방 인테리어에 세련된 느낌을 줍니다.
  - 고른 열 분포로 균일한 조리가 가능하며, 대형 요리를 손쉽게 할 수 있습니다.

### 2. **삼성 BESPOKE 인덕션 NZ63B5056AK**
- **화구 수**: 3구
- **주요 기능**:
  - **듀얼링 기능**: 오른쪽 메인 화구가 28cm로 대형 조리기구도 쉽게 사용할 수 있습니다.
  - **플렉스존 기능**: 다양한 크기의 조리기구를 지원합니다.
- **장점**:
  - 대화구로 인해 큰 냄비나 팬을 사용해도 편리하며, 디자인 또한 모던하여 인테리어와 잘 어울립니다.

### 3. **삼성 인덕션 NZ63D650BXH**
- **화구 수**: 3구
- **주요 기능**:
  - 기본적인 인덕션 기능으로 사용이 간편하며, 다양한 크기의 조리기구를 지원합니다.
- **장점**:
  - 저렴한 가격으로 화이트 디자인을 제공하여 깔끔한 주방 분위기를 연출할 수 있습니다.
  - 화재 방지 기능이 있어 안전하게 사용할 수 있습니다.

이 제품들은 각각의 특성과 장점을 가지고 있어, 여러분의 요리 스타일이나 주방 인테리어에 맞는 선택을 도와줄 것입니다! 🏡🔥

elapsed time: 20.29 seconds
Original Input: 부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘
Web Search Query: 삼성전자 TV 추천
LLM Query: 부모님에게 선물하기에 적합한 삼성전자 TV 모델을 추천해 주세요. 각 모델의 주요 기능과 장점을 포함해서 설명해 주시면 감사하겠습니다.
Bing Grounding 검색 사용: 삼성전자 TV 추천
Existing agent ID: asst_eY1zfMrBH0XcigQZpOrGXmXk
Message created, ID: msg_ukJHH4HJuxuuzBhDAV6yvBV3
Run completed successfully. Status: RunStatus.COMPLETED
Extracted Text Content:
Here are the top three relevant results for "삼성전자 TV 추천":

1. **TV 추천: 삼성전자 TOP 4 리뷰&비교** - Highlights QLED model for performance/value balance 【3:0†source】.
2. **삼성 스마트TV 추천 top10 후기 & 추천** - Details trusted customer feedback on Samsung UHD 4K models【3:2†source】.
3. **TOP 3 삼성 티비 추천 모델 리뷰** - Compares budget-friendly options like the Crystal UHD line for value seekers 【3:8†source】.
Web Search Results: 3


부모님에게 선물하기 적합한 삼성전자 TV 모델을 추천해 드리겠습니다. 아래 두 가지 모델은 사용하기 편리하며, 좋은 화질과 가격대가 적절해 부모님이 사용하시기에 안성맞춤입니다! 📺✨

### 1. 삼성 QLED 70번대 (2024년형)
- **주요 기능**: 
  - **퀀텀프로세서 4K**: 뛰어난 화질 보정 기능으로 다양한 콘텐츠를 더욱 생생하게 감상할 수 있습니다.
  - **120Hz 고주사율 지원**: 스포츠 경기나 고속 장면도 부드럽게 보여줍니다.
  - **FreeSync Premium**: 게임 시 더 나은 화면 품질을 제공합니다.
- **장점**:
  - 색상 표현이 뛰어나 자연 다큐멘터리 같은 고화질 콘텐츠를 시청할 때 최상의 경험을 제공합니다.
  - 크기가 다양해 (55인치부터 85인치까지) 집안 환경에 맞춰 선택할 수 있습니다. 

### 2. 삼성 Crystal UHD 7000라인 (2024년형)
- **주요 기능**: 
  - **4K UHD 해상도**: 선명하고 세밀한 화질로 다양한 OTT 콘텐츠를 즐길 수 있습니다.
  - **음성인식 리모컨 선택 가능**: 편리한 음성 조작 기능이 포함된 모델도 있습니다.
- **장점**:
  - 가격이 매우 합리적이어서 가성비가 뛰어납니다 (65인치 기준 약 100만 원 전후).
  - 화면 크기 옵션이 넓어 (43인치부터 85인치까지) 부모님 방이나 거실에 적합한 사이즈를 선택할 수 있습니다.

이 두 모델 모두 부모님이 사용하시기에 최적의 선택이 될 수 있습니다. 각각의 특징과 장점을 고려하여 선택하시면 좋을 것 같습니다! 😊

...

elapsed time: 20.30 seconds
Original Input: 갤럭시 버즈 이어버드 한쪽을 새로 구매했는데 페어링 어떻게 하나요
Web Search Query: 갤럭시 버즈 이어버드 한쪽 페어링 방법
LLM Query: 갤럭시 버즈 이어버드의 한쪽을 새로 구매했는데, 이 한쪽 이어버드를 기존의 이어버드와 페어링하는 방법을 알려주세요. 필요한 절차와 주의사항도 포함해 주시면 좋겠습니다.
Bing Grounding 검색 사용: 갤럭시 버즈 이어버드 한쪽 페어링 방법
Existing agent ID: asst_eY1zfMrBH0XcigQZpOrGXmXk
Message created, ID: msg_zOXGTwGN0XvNTZATxm5pv3QH
Run completed successfully. Status: RunStatus.COMPLETED
Extracted Text Content:
Here are the top 3 relevant links:

1. 갤럭시 버즈 한쪽 이어버드 페어링 문제 해결 방법 - Yogidamoa.tistory.com 【3:0†source】
2. 갤럭시 버즈 페어링 방법과 웨어러블 앱 안내 - Changwoo.tistory.com 【3:2†source】
3. 갤럭시 버즈 한쪽 연결 안됨 해결법 - How2blog.tistory.com 【3:3†source】
Web Search Results: 3


갤럭시 버즈 이어버드의 한쪽을 새로 구매하셨군요! 🎧✨ 기존 이어버드와 새로 구매한 이어버드를 페어링하는 방법은 다음과 같습니다.

### 페어링 방법

1. **기존 이어버드 초기화**
   - 먼저, 기존 이어버드를 초기화해야 합니다. 이어버드를 케이스에 넣고 뚜껑을 닫습니다.
   - 약 10초 후에 뚜껑을 열고, 이어버드의 터치패드를 7초 이상 눌러줍니다. 이때 LED가 깜박이면서 초기화가 완료됩니다.

2. **새로운 이어버드 준비**
   - 새로 구매한 이어버드도 케이스에 넣고 충전이 완료되었는지 확인합니다.

3. **양쪽 이어버드 페어링 모드로 전환**
   - 기존 이어버드와 새로 구매한 이어버드를 모두 케이스에 넣은 상태에서, 두 이어버드의 뚜껑을 열어 페어링 모드로 전환합니다.

4. **스마트폰에서 연결**
   - 갤럭시 스마트폰의 **Galaxy Wearable 앱**을 열어주세요. 앱이 설치되어 있지 않다면 구글 플레이 스토어에서 다운로드 후 설치해 주세요.
   - 앱 내에서 **디바이스 추가**를 선택하고, 연결할 갤럭시 버즈를 선택합니다.
   - 연결 요청이 뜨면 승인하여 페어링을 완료합니다.

### 주의사항

- **펌웨어 업데이트**: 양쪽 이어버드의 펌웨어가 서로 다를 경우 페어링이 실패할 수 있으니, Galaxy Wearable 앱을 통해 펌웨어를 최신 상태로 유지하는 것이 중요합니다.
- **배터리 확인**: 이어버드의 배터리가 충분한지 확인하세요. 배터리가 부족하면 연결이 원활하지 않을 수 있습니다.
- **초기화 필요성**: 기존 이어버드가 정상적으로 연결되었던 경우, 초기화를 통해 문제를 해결할 수 있습니다. 초기화 후에는 다시 페어링 절차를 따라야 합니다.

이 단계를 따라 하시면 새로운 이어버드를 기존의 이어버드와 성공적으로 연결할 수 있을 것입니다. 필요시 삼성 서비스센터에 문의하는 것도 좋은 방법입니다. 😊

elapsed time: 19.70 seconds
Original Input: 삼성전자 S25 무게가 S24와 비교 했을때 얼마나 차이나
Web Search Query: 삼성전자 S25 S24 무게 비교
LLM Query: 삼성전자 S25와 S24의 무게 차이를 알려주세요. 각각의 모델 무게를 비교하고, 어떤 모델이 더 가벼운지 명확히 설명해 주세요.
Bing Grounding 검색 사용: 삼성전자 S25 S24 무게 비교
Existing agent ID: asst_eY1zfMrBH0XcigQZpOrGXmXk
Message created, ID: msg_ykP32oVziutQ8uNPCxIHTFex
Run completed successfully. Status: RunStatus.COMPLETED
Extracted Text Content:
The top 3 most relevant results are:

1. **"갤럭시 S25 울트라 vs S24 울트라 비교"** on Kongnamu Tistory (details about weight and specs) 【3:0†source】.
2. **"갤럭시 S25 S24 S23 S22 차이점 비교"** on TaeinPapa Tistory (lists weight changes alongside dimensions) 【3:1†source】.
3. **"갤럭시 S24 vs S25 스펙 비교표"** on Its-Memory Tistory (comparison table highlighting weight differences) 【3:3†source】.
Web Search Results: 3


갤럭시 S25와 S24의 무게 차이를 살펴보면 다음과 같습니다. 📱

- **갤럭시 S24**: 232g
- **갤럭시 S25**: 218g

S25는 S24보다 14g 가벼워졌습니다. 따라서 갤럭시 S25가 더 가벼운 모델입니다. 이는 휴대성을 더욱 향상시켜 사용자가 보다 편리하게 스마트폰을 사용할 수 있게 합니다. 가벼운 무게는 장시간 사용 시 손목 부담을 줄여주는 효과도 있죠! 😄

elapsed time: 13.50 seconds
